<a href="https://colab.research.google.com/github/seki-shu/Group15_Multimodal/blob/Robosuite/Robosuite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Robosuiteの基本

## Import libraries

In [ ]:
%pip install robosuite

import robosuite as suite
import time
import pandas as pd
import matplotlib.pyplot as plt
import moviepy.editor as mpy
from base64 import b64encode
from IPython.display import HTML
from google.colab import files
import uuid, shutil
from typing import List
import numpy as np

import os

## シミュレーション環境の初期化

In [ ]:
# 環境の初期化
env = suite.make(
    env_name="Lift",  # タスク名（Liftタスク）
    robots="Panda",   # 使用するロボット（Pandaロボット）
    has_renderer=False,  # リアルタイムレンダリングを無効にする
    use_camera_obs=True  # カメラ観測を使用する
)
# 環境のリセット
obs = env.reset()


## アクションの実行

In [ ]:
# アクションスペックを取得
action_spec = env.action_spec

# アクションの最小値と最大値を取得
low, high = action_spec

# ランダムなアクションを生成
action = np.random.uniform(low, high)

# アクションを実行し、次の状態、報酬、終了フラグ、情報を取得
obs, reward, done, info = env.step(action)

# 実行結果を表示
print("Observation:", obs.keys())
print("Observation:", obs["cube_quat"])
plt.imshow(obs["agentview_image"]); plt.show()
print("Reward:", reward)
print("Done:", done)

# マルチモーダル情報を扱えるRobosuite

## Import libraries

In [ ]:
%pip install robosuite
%pip install gymnasium

import torch
from torch import nn
from torch.nn import functional as F
import torch.distributions as td
from torch.distributions import Normal, Categorical, OneHotCategorical, OneHotCategoricalStraightThrough, Bernoulli
from torch.distributions import kl_divergence
import robosuite as suite
from robosuite.wrappers.gym_wrapper import GymWrapper

import numpy as np

import os

## シミュレーション環境の初期化

In [ ]:
# 環境の初期化
env = suite.make(
    env_name="PickPlace",  # タスク名
    robots="Panda",   # 使用するロボット
    has_renderer=False,  # リアルタイムレンダリング
    has_offscreen_renderer=True,  # オフスクリーンレンダリング
    use_camera_obs=True,  # カメラ情報を使用
    camera_names=["frontview", "robot0_eye_in_hand"],  # 複数カメラ
    camera_heights=84,  # カメラ画像の高さ
    camera_widths=84,  # カメラ画像の幅
    camera_depths=False,  # 深度画像をオフ
    use_object_obs=True,  # オブジェクト情報を使用
    use_force_sensor=True
)

# GymWrapperを使ってOpenAI Gym互換インターフェースに変換
# env = GymWrapper(env)

# 環境のリセット
obs = env.reset()
# 環境の初期状態（力覚データを含む場合、"robot0_eef_force"を確認）
print("Observation:", obs.keys())  # 力覚データが含まれていることを確認

## マルチモーダルデータの取得

In [ ]:
low, high = env.action_spec

# ランダムなアクションを生成
action = np.random.uniform(low=low, high=high, size=low.shape)

# 環境を1ステップ進める
obs, reward, done, info = env.step(action)

# 取得可能な情報を表示
print("Camera Observation:", obs["frontview_image"].shape)  # カメラ画像
# print("Force Sensor Data:", obs["robot0_eef_force"])  # 力覚データ
print("Object Position:", obs["cube_pos"])  # オブジェクトの位置